In [0]:
try:
    dbutils.fs.unmount("/mnt/mf-production")
except Exception as e:
    pass  

/mnt/mf-production has been unmounted.


In [0]:
dbutils.fs.mount(
    source = "wasbs://mf-production@mutualfundstoragecdac.blob.core.windows.net",
    mount_point = "/mnt/mf-production",
    extra_configs={
        "fs.azure.sas.mf-production.mutualfundstoragecdac.blob.core.windows.net": dbutils.secrets.get(scope="mutual-fund-keyvalut-secrets", key="storage-sas-prod-sas")
    })

daily_path = "/mnt/mf-production/daily_extracts"


In [0]:
from datetime import datetime
import pytz
import pyarrow.parquet as pq
import pandas as pd
from pyspark.sql.functions import col
from pyspark.sql.types import DecimalType

today = datetime.now(pytz.timezone("Asia/Kolkata"))
today_path = f"{daily_path}/{today.year:04d}/{today.month:02d}/{today.day:02d}"

pandas_frames = []

try:
    files = dbutils.fs.ls(today_path)
    if not files:
        print(f"No files found at {today_path}")
        files = []
    else:
        print(f"Found {len(files)} files at {today_path}")
        
except Exception as e:
    print(f"Path not found or inaccessible: {str(e)}")
    files = []

if files:
    try:
        for file in files:
            local_path = file.path.replace("dbfs:/", "/dbfs/")
            table = pq.read_table(local_path)
            pandas_frames.append(table.to_pandas())

        full_df = pd.concat(pandas_frames, ignore_index=True)

        if 'insert_date' in full_df.columns:
            full_df['insert_date'] = pd.to_datetime(full_df['insert_date'], unit='ns')

        spark_df = spark.createDataFrame(full_df)
        spark_df = spark_df.withColumn("nav", col("nav").cast(DecimalType(18, 6)))

        print("Spark DataFrame created and transformed successfully.")

    except Exception as e:
        print(f"Error during file processing or DataFrame transformation: {str(e)}")


Found 17 files at /mnt/mf-production/daily_extracts/2025/07/26
Spark DataFrame created and transformed successfully.


In [0]:
try:
    connection_properties = {
        "user": 'cdac',
        "password": dbutils.secrets.get(scope="mutual-fund-keyvalut-secrets", key="storage-database-secret"),
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }

    spark_df.write \
        .mode("append") \
        .jdbc(url=dbutils.secrets.get(scope="mutual-fund-keyvalut-secrets", key="storage-database-jdbc"), table='ActiveMutualFundsNavs', properties=connection_properties)

except Exception as e:
    print(f"Error writing to SQL Server: {str(e)}")

In [0]:
try:
    connection_properties = {
        "user": 'cdac',
        "password": dbutils.secrets.get(scope="mutual-fund-keyvalut-secrets", key="storage-database-secret"),
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }

    spark_df.write \
        .mode("append") \
        .jdbc(url=dbutils.secrets.get(scope="mutual-fund-keyvalut-secrets", key="storage-database-jdbc"), table='Prod.ActiveMutualFundsNavs', properties=connection_properties)

except Exception as e:
    print(f"Error writing to SQL Server: {str(e)}")

In [0]:
try:
    dbutils.fs.unmount("/mnt/mf-production")
except Exception as e:
    print("Failed to unmount the mount point: ", e)
    
print("Unmounted successfully")

/mnt/mf-production has been unmounted.
Unmounted successfully
